In [2]:
import numpy as np

from data_path import DATA_PATH_SROK8C

In [3]:
file = open(DATA_PATH_SROK8C+"20046.dat")

In [4]:
file.readline()

'20046 1965 12 31 18 1966  1  1 21  1 23  4 20    7 0  0 0  0 0  0 0  0 0  0 0  0 0  0 0      5 0    7  0 0  2 0   0 0  0 0 0    7 0        7 -39.0 0       7       7       7       7 -36.1 0       7 0       7       7       7       7       7 1  79 0    0.10 0 1 -38.0 0 1012.6 0 1015.7 0  2 0  0.2 0\n'

In [7]:
int("01")

1

In [ ]:
VALID_QUALITIES = [0, 1, 2]

def dictFromSrok8cDatFile(stationSynopticIndex: int):
    d = {
        "GMT year" : list(),
        "GMT month" : list(),
        "GMT day" : list(),
        "GMT period" : list(),
        "Year of source (local)" : list(),
        "Month of source (local)" : list(),
        "Day of source (local)" : list(),
        "Source period" : list(),
        "Local time" : list(),
        "Horizontal visibility (km)" : list(),
        "Total cloud amount (points)"
    }

def hrzVsbltInKm(code: str, quality: int):
    res = np.nan
    if quality in VALID_QUALITIES:
        if code == "00":
            res = 0.99
        elif code in genCodes(1, 50+1):
            res = float(int(code)) / 10.0
        elif code in genCodes(56, 80+1):
            res = float(code) - 50.0
        elif code in genCodes(81, 88+1):
            res = (float(code) - 76.0) * 5.0
        elif code == "89":
            res = 70.01
        elif code == "90":
            res = 0.0499
        elif code == "91":
            res = 0.05
        elif code == "92":
            res = 0.2
        elif code == "93":
            res = 0.3
        elif code == "94":
            res = 1.0
        elif code == "95":
            res = 2.0
        elif code == "96":
            res = 4.0
        elif code == "97":
            res = 10.0
        elif code == "98":
            res = 20.0
        elif code == "99":
            res = 50.01    
    return res

def genCodes(start, end):
    res = list()
    if end > 99:
        return np.nan
    for i in range(start, end):
        if i < 10:
            res += [f"0{i}"]
        else:
            res += [f"{i}"]
    return res